# Eat Safe, Love

## Notebook Set Up

In [ ]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our database
db.list_collection_names()

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

In [ ]:
establishments.find_one()

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}
all_documents = establishments.find_one(query)
# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(count)

In [ ]:

# Display the first document in the results using pprint
first_result = establishments.find(query)
pprint(first_result[0])

In [ ]:
# Convert the result to a Pandas DataFrame

result = list(first_result)
df = pd.DataFrame(result)


# Display the number of rows in the DataFrame
print(len(df))


In [ ]:
# Display the first 10 rows of the DataFrame
df.head(10)

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
establishments.find_one()

In [ ]:
import re

# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {"LocalAuthorityName": {"$regex": re.compile(r"London", re.IGNORECASE)}, "RatingValue": {"$gte": 4}}
results = establishments.find(query)


# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(count)



In [ ]:
# Display the first document in the results using pprint
first_result = establishments.find(query)
pprint(first_result[0])

In [ ]:
# Convert the result to a Pandas DataFrame
result = list(first_result)
df2 = pd.DataFrame(result)


# Display the number of rows in the DataFrame
print(len(df2))





In [ ]:
# Display the first 10 rows of the DataFrame
df2.head(10)

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
result = establishments.find_one() 

latitude = result["geocode"]["latitude"]
longitude = result["geocode"]["longitude"]

print("Latitude data type:", type(latitude))
print("Longitude data type:", type(longitude))

In [ ]:
result = establishments.find_one({"BusinessName": "Penang Flavours"})


lat = result["geocode"]["latitude"]
long = result["geocode"]["longitude"]
print("Latitude:", lat)
print("Longitude:", long)

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = lat
longitude = long

query = {'RatingValue': 5, 'geocode.latitude': {'$lte': (latitude + degree_search), '$gte': (latitude - degree_search)}, 
         'geocode.longitude': {'$lte': (longitude + degree_search), '$gte': (longitude - degree_search)}}
sort_results = establishments.find(query).sort("scores.Hygiene", -1).limit(5)

# Print the results
for result in sort_results:
    pprint(result)

In [ ]:
# Convert the result to a Pandas DataFrame
df3 = pd.DataFrame(establishments.find(query).sort("scores.Hygiene", 1).limit(5))

# Display the number of rows in the DataFrame
print(f"There are {len(df3)} rows in the DataFrame.")

# Display the first 10 rows of the DataFrame
df3.head()

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
match_query = {'$match': {'scores.Hygiene': 0}}
group_query = {'$group': {'_id': '$LocalAuthorityName', 'count': {'$sum': 1}}}
sort_query = {'$sort': {'count': -1}}

pipeline = [match_query, group_query, sort_query]

results = list(establishments.aggregate(pipeline))

# Print the number of documents in the result
print(f"There are {len(results)} results")

# Print the first 10 results
pprint(results[0:10])

In [ ]:
# Convert the result to a Pandas DataFrame
df4 = pd.json_normalize(results)

# Display the number of rows in the DataFrame
print(f"There are {len(df4)} rows in the DataFrame.")

# Display the first 10 rows of the DataFrame
df4.head(10)
